### PROJECT ABSTRACT 

With all the code computed on other notebookes:

 **1- Spotify API** - where I did my first connection to the API, providing it with like and disliked songs, as well as my musical journey data.

 **2- EDA** - where I explored all Spotify features based on the two groups in study.

 **3- My Musical Journey** - also a kind of EDA notebook. However, in this one I analysed my top year playlists to evaluate the evolution of my music taste from 2016 to 2019. 

 **4- Supervised ML** - tested multiple suprevised learning classifiers getting to a 86% Accuracy with Random Forest. I have now a pretty good algorithm that aimns to detect if I like a song or not, as well as the prob of that event happening.

 **5- Test on other's playlists** - used the best algorithm evaluated on ML notebook and applied to outside data - songs from friend's playlists - meaning neither data from the traning nor the test set.

 **6- Unsupervised Learning** - without any song labeling, used the whole dataset (liked + disliked songs) to extract knowledge based on any patterns: Clustering, Dimensionality Reduction.

Finally, in this script I aim to produce a final product where the user can input his/hers/theirs **(public) playlsit URI** and get an  **Exploratory Data Analysis on the songs' features, as well as new song recomendations.** This will be produced based on Unsupervised Learning techniques, since we can assume the shared playlist will be a collection of liked songs and therefore we can't apply labelling techniques.

In [86]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
import json
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt
import seaborn as sns
import seaborn as sb
%matplotlib inline
import plotly.express as px

import plotly.figure_factory as ff

from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from yellowbrick.cluster import KElbowVisualizer
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
from mpl_toolkits.mplot3d import Axes3D
from scipy.spatial.transform import Rotation as R
import scipy.cluster.hierarchy as shc

import statsmodels.api as sm
from scipy import stats
from scipy.stats import ttest_1samp, ttest_rel, ttest_ind
from IPython.display import display, Image, SVG, Math, YouTubeVideo


In [13]:
username = 'britopereiraa'
client_id ='17a3ed14ab434b33b28f19f8bea3337d'
client_secret = '7f848f9d2a024b15be5fa776b4dc7a05'
redirect_uri = 'http://localhost:7777/callback'
scope = 'user-read-recently-played'

token = util.prompt_for_user_token(username=username, 
                                   scope=scope, 
                                   client_id=client_id,   
                                   client_secret=client_secret,     
                                   redirect_uri=redirect_uri)

client_credentials_manager = SpotifyClientCredentials(client_id, client_secret) 
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [76]:
def master_function(username, uri):
    uri = playlist_uri    
    username = username
    playlist_id = uri
    results = {'items':[]}

    for n in range(0,3000,100):
        new = sp.user_playlist_tracks(username, playlist_id,  offset = n)
        results['items'] += new['items']

        playlist_tracks_data = results
        playlist_tracks_id = []
        playlist_tracks_titles = []
        playlist_tracks_artists = []

        for track in playlist_tracks_data['items']:
            playlist_tracks_id.append(track['track']['id'])
            playlist_tracks_titles.append(track['track']['name'])

        #adds a list of all artists involved in the song to the list of artists for the playlist
            for artist in track['track']['artists']:
                artist_list = []
                artist_list.append(artist['name'])
            playlist_tracks_artists.append(artist_list[0])
    
    #some tracks might have none id
    playlist_tracks_id = list(filter(None, playlist_tracks_id)) 
    
    df = pd.DataFrame([])
    for i in range(0, len(playlist_tracks_id)):
        features = sp.audio_features(playlist_tracks_id[i])
        features_df = pd.DataFrame(features)
        df = df.append(features_df)
       

    df['title'] = playlist_tracks_titles
    #features_df['first_artist'] = playlist_tracks_first_artists
    df['main_artist'] = playlist_tracks_artists
    #features_df = features_df.set_index('id')
    df = df[['id', 'title', 'main_artist',
                               'danceability', 'energy', 'key', 'loudness',
                               'mode', 'acousticness', 'instrumentalness',
                               'liveness', 'valence', 'tempo',
                               'duration_ms', 'time_signature']]

    return df

### Get the all songs playlist: 

A playlist with over 5000 songs of all Genre 

In [88]:
features = ['danceability', 'energy', 'key','loudness', 'mode', 'acousticness', 'instrumentalness', 'liveness','valence', 'tempo']

#import dataset created in other jupyter notebook
all_songs = pd.read_csv('../data/all_songs.csv')

#features scaling 
scaler = StandardScaler()
scaler.fit(all_songs[features])

all_songs_features_scaled = scaler.transform(all_songs[features])

In [89]:
#label the data into clusters

kmean = KMeans(n_clusters=9)
kmean.fit(all_songs_features_scaled)

pred = kmean.labels_
all_songs['labels'] = pred

all_songs.head()

,id,title,main_artist,danceability,energy,key,loudness,mode,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,labels
0,2aQ8NyRJcsQFgb8jwUil90,Que Bandeira,Evinha,0.755,0.537,1,-10.679,1,0.75600,0.000003,0.3550,0.811,93.393,149420,4,6
1,6d4KqeYESkFHMirUgE1Aa3,Não Vá,Tim Maia,0.658,0.741,2,-10.495,1,0.04430,0.001280,0.0603,0.928,103.283,307037,4,6
2,2OoOgl80tDTElnbMYfhJyH,Reggins,The Blackbyrds,0.645,0.383,5,-16.549,0,0.00495,0.611000,0.0727,0.943,97.238,244800,4,5
3,5voWjUklIhjnNQU5aZCIm4,Dreaming About You,The Blackbyrds,0.744,0.292,4,-16.592,0,0.26500,0.095900,0.0738,0.881,96.366,352400,4,7
4,2jHa1UWWuouaoFqE8I7W77,Never Let Her Go,David Gates,0.576,0.399,8,-14.764,1,0.23300,0.000022,0.1040,0.621,84.049,188813,4,2


#### Test with a random playlist

In [77]:
#input variables
playlist_uri ="2ERd3YQjKnGSEyInrwYPRh"
username = "João Fanha"

In [78]:
df = master_function(username,playlist_uri)
df.head()

,id,title,main_artist,danceability,energy,key,loudness,mode,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,1CsCuwscKV2tYqLeG0p8Q6,Toyota Man,Neon Indian,0.563,0.864,7,-5.313,1,0.0685,0.000005,0.5160,0.896,93.901,252883,4
0,6wYTcgNMqPZw2hybRMTEQf,Timebends,Deerhunter,0.369,0.796,10,-8.236,1,0.2250,0.345000,0.2630,0.461,142.902,758151,4
0,0uYRKUD994hbQi3JLSt22K,The Left,Duckwrth,0.808,0.703,7,-7.772,1,0.0636,0.005150,0.1050,0.392,90.003,263876,4
0,2uJT2q88v5RYhn5XRhAryM,Netflix,Malxs,0.720,0.504,4,-7.658,1,0.1890,0.000000,0.1180,0.377,92.001,159130,4
0,2pDKE8Q40TDGPl1O11DKKn,Shameika,Fiona Apple,0.375,0.776,8,-6.767,1,0.1540,0.000082,0.0925,0.647,186.814,248603,4


In [ ]:
data['labels'].value_counts()